# This cell configures the spark session - Do not change

In [1]:
%%configure -f
{
"conf": {
     "spark.rpc.message.maxSize": 1024,
     "spark.kryoserializer.buffer.max": "256m"
   }
}

StatementMeta(, 38, -1, Finished, Available)

# These are the parameters that need to be changed to your values

In [2]:
# The input file name 
input_filename = 'abfss://share@[accountname].dfs.core.windows.net/sport_articles.csv'
# The output directory where the output file will be written to
output_directory = 'abfss://share@[accountname].dfs.core.windows.net/sport_articles/output/'
# The name of the output file
output_filename = 'sports_heuristics.csv'



StatementMeta(small, 38, 2, Finished, Available)

In [3]:
from pyspark import SparkContext, SparkConf
import sys
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
import ntpath
import os
from  pyspark.sql.functions import input_file_name
import numpy as np
from pyspark.sql.functions import regexp_extract,col
from pyspark.sql.types import ArrayType,StringType, FloatType, BooleanType
from flashtext import KeywordProcessor


df = spark.read.load(input_filename, header=True, format='csv')

df.show()



StatementMeta(small, 38, 3, Finished, Available)

+---+-----------------+--------------------+
|_c0|         filename|                text|
+---+-----------------+--------------------+
|  0|  football054.txt|"palace threat ov...|
|  1|  football040.txt|irish finish with...|
|  2|  football068.txt|"cole faces lengt...|
|  3|  football097.txt|"tottenham bid &#...|
|  4|  football083.txt|"thompson says ge...|
|  5|  football256.txt|"uganda fa suspen...|
|  6|  football242.txt|"pennant guilty o...|
|  7|   cricket030.txt|"denness leads lu...|
|  8|   cricket024.txt|"indias top six s...|
|  9|   cricket018.txt|"streak happy wit...|
| 10|  football120.txt|mansfield 0-1 ley...|
| 11|     rugby029.txt|wales win in rome...|
| 12|  football134.txt|"houllier praises...|
| 13|     rugby015.txt|"ref stands by sc...|
| 14|  football108.txt|"santini resigns ...|
| 15|     rugby001.txt|"hodgson shoulder...|
| 16|atheletics014.txt|"mcilroy aiming f...|
| 17|    tennis019.txt|"venus stunned by...|
| 18|    tennis031.txt|"moya fights back...|
| 19|athel

# Apply some simple search rules

In [10]:
list_of_sports = ["athletics", "football", "rugby", "tennis", "cricket"]

keyword_processor = KeywordProcessor()

for sport in list_of_sports:
    keyword_processor.add_keyword(sport)


def classify(row):              
    keywords_found_top = keyword_processor.extract_keywords(row, span_info=True)

    if len(keywords_found_top) > 0:
        found = sorted(keywords_found_top, reverse=False, key=lambda x:x[1])[0][0]
        # Unfortunately we have spelling mistake in a label so let's apply it
        if found == "athletics":
            found = "atheletics"
    else:
        found = ""
    return found

udf_classify = udf(classify, StringType())
df = df.withColumn("simple_sport_classification", udf_classify(df.text))

StatementMeta(small, 38, 10, Finished, Available)

# Generate the actual labels from the file names

In [11]:
import re
def find_first_digit(s1):

    return_index = -1
    return_result = ""
    m = re.search(r"\d", s1)
    
    if m:
        return_index = m.start()
        return_result = s1[:m.start()]
              
    return return_result

udf_get_label = udf(find_first_digit, StringType())

df = df.withColumn("actual_sport_classification", udf_get_label(df.filename))

StatementMeta(small, 38, 11, Finished, Available)

In [12]:
display(df)

StatementMeta(small, 38, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3b968899-c462-42e6-96b2-d7a4f1a621c0)

In [13]:
y_true = df.select("actual_sport_classification").rdd.flatMap(lambda x: x).collect()
y_pred = df.select("simple_sport_classification").rdd.flatMap(lambda x: x).collect()

StatementMeta(small, 38, 13, Finished, Available)

# Let's evaluate

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

print(len(y_pred), len(y_true))


print(classification_report(y_true, y_pred))

StatementMeta(small, 38, 14, Finished, Available)

737 737
              precision    recall  f1-score   support

                   0.00      0.00      0.00         0
  atheletics       0.97      0.32      0.48       101
     cricket       0.98      0.40      0.57       124
    football       1.00      0.19      0.32       265
       rugby       0.96      0.35      0.52       147
      tennis       1.00      0.31      0.47       100

    accuracy                           0.29       737
   macro avg       0.82      0.26      0.39       737
weighted avg       0.99      0.29      0.44       737

